In [1]:
import numpy as np
import pandas as pd
import json
# haandle data
import json
import operator
import collections
from sklearn import svm
from sklearn.model_selection  import train_test_split

In [2]:
# load json file
train_data = pd.read_json('./data/train.json')
test_data = pd.read_json('./data/test.json')

In [3]:
train_data.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [4]:
# get unique ing and cui
def get_unique_ingredients():
    unique_ing = set()
    #     df['ingredients'].map(lambda x: [ unique_ing.add(ing) for ing in x])
    train_data['ingredients'].map(lambda x: [ unique_ing.add(ing) for ing in x])
    test_data['ingredients'].map(lambda x: [ unique_ing.add(ing) for ing in x])
    return list(unique_ing)

def get_unique_cuisines(df):
    return df['cuisine'].unique().tolist()


unique_ingredients = get_unique_ingredients()
unique_cuisines = get_unique_cuisines(train_data)

In [5]:
print(f'{len(unique_cuisines)}:{unique_cuisines}')
print(f'{len(unique_ingredients)}:{unique_ingredients[:10]}')

20:['greek', 'southern_us', 'filipino', 'indian', 'jamaican', 'spanish', 'italian', 'mexican', 'chinese', 'british', 'thai', 'vietnamese', 'cajun_creole', 'brazilian', 'french', 'japanese', 'irish', 'korean', 'moroccan', 'russian']
7137:['soy nuts', 'deli ham', 'sponge', 'loin', 'whole garam masala', 'molasses', 'scape pesto', 'tostada shells', 'bread ciabatta', 'mrs. dash seasoning mix']


In [6]:
number_of_ingredients = len(unique_ingredients)
print("number of ingredients:", number_of_ingredients)
number_of_train_dishes = len(train_data)
print("number of trainning dishes:", number_of_train_dishes )
number_of_test_dishes = len(test_data)
print("number of testing dishes:", number_of_test_dishes)

number of ingredients: 7137
number of trainning dishes: 39774
number of testing dishes: 9944


In [7]:
def get_max_min_ing(df):
    len_ings = []
    df['ingredients'].map(lambda x: len_ings.append(len(x)))
    return max(len_ings), min(len_ings)

train_data_max_ing, train_data_min_ing = get_max_min_ing(train_data)
test_data_max_ing, test_data_min_ing = get_max_min_ing(test_data)

In [8]:
print(f'train max: {train_data_max_ing}')
print(f'train min: {train_data_min_ing}')
print(f'test max: {test_data_max_ing}')
print(f'test min: {test_data_min_ing}')

train max: 65
train min: 1
test max: 50
test min: 1


In [9]:
def encode_ingredients(df):
    df['ingredients_encode'] = df['ingredients'].map(lambda x: [unique_ingredients.index(ing)+1 for ing in x])
    return df

train_data = encode_ingredients(train_data)
print('done train_data')
test_data = encode_ingredients(test_data)
print('done test_data')

done train_data
done test_data


In [10]:
train_data.head()

,cuisine,id,ingredients,ingredients_encode
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...","[2375, 5639, 2403, 5923, 6783, 3436, 6653, 581..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...","[3297, 6212, 654, 1480, 3807, 2788, 6411, 6468..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...","[6411, 6783, 654, 271, 2732, 5581, 4164, 4971,..."
3,indian,22213,"[water, vegetable oil, wheat, salt]","[3261, 7060, 6499, 654]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...","[2475, 4324, 3798, 6351, 3715, 1626, 4978, 191..."


In [11]:
def encode_cuisine(df):
    df['cuisine_encode'] = df['cuisine'].map(lambda x: unique_cuisines.index(x))
    return df
train_data = encode_cuisine(train_data)

In [12]:
def encode_cuisine(df):
    df['cuisine_onehot'] = df['cuisine_encode'].map(lambda x: [1 if i==x else 0 for i in range(len(unique_cuisines))])
    return df

train_data = encode_cuisine(train_data)

In [13]:
train_data.tail()

,cuisine,id,ingredients,ingredients_encode,cuisine_encode,cuisine_onehot
39769,irish,29109,"[light brown sugar, granulated sugar, butter, ...","[3633, 3903, 1914, 1516, 5917, 1207, 6163, 460...",16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
39770,italian,11462,"[KRAFT Zesty Italian Dressing, purple onion, b...","[6815, 3436, 3485, 2156, 7120, 309, 3750]",6,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
39771,irish,2238,"[eggs, citrus fruit, raisins, sourdough starte...","[6411, 5615, 3037, 642, 2356, 1747, 3269, 5723...",16,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
39772,chinese,41882,"[boneless chicken skinless thigh, minced garli...","[2056, 547, 3994, 3657, 1183, 2045, 597, 186, ...",8,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
39773,mexican,2362,"[green chile, jalapeno chilies, onions, ground...","[4797, 4037, 3715, 3807, 654, 6874, 469, 5923,...",7,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
l = [1,2,3]
l.extend([i for i in range(1,10)])
print(l)

[1, 2, 3, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [23]:
# helper 
def dish_list_add(dish):
    # input: (train max) 65
    number_of_non_ingredients = 65 - len(dish)
    extended_list = [0 for i in range(number_of_non_ingredients)]
    encoded_dish = dish + extended_list
    return encoded_dish 

def normalize_ingredients(df):
    # input: (train max) 65
    df['ingredients_features'] = df['ingredients_encode'].map(dish_list_add)
    return df

train_data = normalize_ingredients(train_data)
test_data  = normalize_ingredients(test_data)

In [21]:
train_data.head()

,cuisine,id,ingredients,ingredients_encode,cuisine_encode,cuisine_onehot,ingredients_features
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes...","[2375, 5639, 2403, 5923, 6783, 3436, 6653, 581...",0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2375, 5639, 2403, 5923, 6783, 3436, 6653, 581..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g...","[3297, 6212, 654, 1480, 3807, 2788, 6411, 6468...",1,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3297, 6212, 654, 1480, 3807, 2788, 6411, 6468..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g...","[6411, 6783, 654, 271, 2732, 5581, 4164, 4971,...",2,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[6411, 6783, 654, 271, 2732, 5581, 4164, 4971,..."
3,indian,22213,"[water, vegetable oil, wheat, salt]","[3261, 7060, 6499, 654]",3,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[3261, 7060, 6499, 654, 0, 0, 0, 0, 0, 0, 0, 0..."
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe...","[2475, 4324, 3798, 6351, 3715, 1626, 4978, 191...",3,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[2475, 4324, 3798, 6351, 3715, 1626, 4978, 191..."


In [85]:
features = np.array(train_data['ingredients_features'])
for indx, feature in enumerate(features):
    features[indx] = np.array(feature)
target = np.array(train_data['cuisine_onehot'])
for indx, targ in enumerate(target):
    target[indx] = np.array(targ)
    

In [86]:
train_X, test_X, train_y, test_y = train_test_split(features, target, random_state=0)

In [135]:
train_X[0]

array([3807, 6531, 1342, 5923, 4257,  798, 2020, 1716,  654,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [88]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout

In [141]:
# define model 
model = Sequential()
model.add(Dense(32, input_dim=65))
model.add(Dense(32, activation=tf.nn.relu))
model.add(Dense(1, activation=tf.nn.softmax))

In [142]:
model.compile(optimizer='adam', 
             loss = 'categorical_crossentropy', 
             metrics=['accuracy'])

In [143]:
batchsize = 100
epochs = 5
# model.fit(train_X, train_y, epochs=epochs, batch_size=batchsize, validation_data=(test_X, test_y))
model.fit(train_X, train_y)

ValueError: Error when checking input: expected dense_66_input to have shape (65,) but got array with shape (1,)

In [132]:
model.save('./whats_cooking_ingredients_as_features.model')

# Submit model

In [54]:
predictions = model.predict(test_model_matrix)

In [55]:
ss = pd.read_csv('./data/sample_submission.csv')
ss.head()

,id,cuisine
0,35203,italian
1,17600,italian
2,35200,italian
3,17602,italian
4,17605,italian


In [56]:
written_preds = [raw_cuisines[np.argmax(pred)] for pred in predictions]

In [57]:
ss['cuisine'] = written_preds

In [58]:
ss.to_csv('./data/submission.csv', index=False)